In [8]:
import usb.core
import usb.util

EP_IN  = 0x82
EP_OUT = 0x2

In [14]:
# find FT600
dev = usb.core.find(idVendor=0x0403, idProduct=0x601e)
for config in dev:  
    # The device was getting "Err 16 busy" on my ubuntu
    for i in range(config.bNumInterfaces):
        if dev.is_kernel_driver_active(i):
            dev.detach_kernel_driver(i) 
dev.reset()

try:
    dev.set_configuration()
except:
    print("Already connected")
EP = []

for cfg in dev:
    for iface in cfg:
        for ep in iface:
            ep_dir = usb.util.endpoint_direction(ep.bEndpointAddress)
            ep_type = usb.util.endpoint_type(ep.bmAttributes)
            EP.append(ep)
            #print(ep)

In [15]:
EP

[<ENDPOINT 0x1: Bulk OUT>,
 <ENDPOINT 0x81: Interrupt IN>,
 <ENDPOINT 0x2: Bulk OUT>,
 <ENDPOINT 0x82: Bulk IN>]

In [13]:
k = dev.write(0x02, b'\x12\x34\x56\x78' * (4096//2) )

USBTimeoutError: [Errno 110] Operation timed out

In [43]:
dev.read(0x82, 4096)  #<ENDPOINT 0x1: Bulk OUT>,

USBTimeoutError: [Errno 110] Operation timed out

# Setup FTDI side

In [1]:
import sys;
import select;
import socket;
import time;
import os;
import random;
from time import sleep;

In [2]:
import ftd3xx
import sys
import ftd3xx._ftd3xx_linux as _ft

In [3]:
numDevices = ftd3xx.createDeviceInfoList()
if (numDevices == 0):
    print("ERROR: No FTD3XX device is detected.");
else:
    print(numDevices," Device")
devList = ftd3xx.getDeviceInfoList()
print(devList)
# Just open the first device (index 0)
devIndex = 0;
D3XX = ftd3xx.create(devIndex, _ft.FT_OPEN_BY_INDEX);  

1  Device


In [4]:
cfg = D3XX.getChipConfiguration();
print("CLK: ",cfg.FIFOClock)
print("Mode: ",cfg.FIFOMode)
print("FlashEEPROMDetection: ",cfg.FlashEEPROMDetection)
VP = D3XX.getVIDPID()
print("VID/PID:",hex(VP[0]), hex(VP[1]))

CLK:  1
Mode:  0
FlashEEPROMDetection:  16
VID/PID: 0x60 0x1e


In [5]:
# Reset the FT600 to make sure starting fresh with nothing in FIFOs
D3XX.resetDevicePort(); # Flush
D3XX.close();

In [32]:

D3XX = ftd3xx.create(devIndex, _ft.FT_OPEN_BY_INDEX);

# check if USB3 or USB2
devDesc = D3XX.getDeviceDescriptor();
bUSB3 = devDesc.bcdUSB >= 0x300;

# validate chip configuration
cfg = D3XX.getChipConfiguration();
cfg.FIFOClock

AttributeError: 'NoneType' object has no attribute 'resetDevicePort'

In [67]:
D3XX.setChipConfiguration(cfg);

AttributeError: 'NoneType' object has no attribute 'setChipConfiguration'

In [70]:
desired_freq = 1;# 1=66 MHz, 0=100 MHz
if ( cfg.FIFOClock != desired_freq ):
    print("Changing FT600 Frequency");
    cfg.FIFOClock = desired_freq;
    D3XX.setChipConfiguration(cfg);
D3XX.resetDevicePort(); # Flush
D3XX.close();

AttributeError: 'NoneType' object has no attribute 'resetDevicePort'

In [69]:
devIndex = 0;
D3XX = ftd3xx.create(devIndex, _ft.FT_OPEN_BY_INDEX);
cfg = D3XX.getChipConfiguration();
cfg.FIFOClock

AttributeError: 'NoneType' object has no attribute 'getChipConfiguration'

In [22]:
D3XX.resetDevicePort(); # Flush
D3XX.close();